In [1]:
%%capture
import sys
!{sys.executable} -m pip install requests pandas geopy bs4 folium sklearn matplotlib

In [2]:
import folium
import requests
import pandas as pd
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
import json
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [3]:
# Setting credentials required by foursquare
CLIENT_ID = 'DEPKYDHCHC40DUDYBECM3Y1GYBGOFGRTODGR4QLNQ5VPNLBU'
CLIENT_SECRET = 'FWKDHU4URB1WHQI5IKQ5QN4RMLEWOASA34AHLW3YNA4KCGH5'
VERSION = '20190808'

<h4>Scrape list of areas in Pune city from "makemytrip" website</h4>

In [4]:
# Function to scrape data from website and convert it into dataframe
geolocator = Nominatim(user_agent="coursera agent")
def create_areas_data_frame(): 
    with open('area_index.html') as f:
        soup = BeautifulSoup(f, parser='lxml')

    city_areas = soup.find_all('div', attrs={'class': 'city-listing'})
    areas_dict = {'Area': [], 'Latitude': [], 'Longitude': []}
    for areas in city_areas:
        area_cat = areas.find('p', attrs={'class': 'key-values'})
        for area_parsed in area_cat.find_all('a'):
            area_short = area_parsed.text.split(',')[0]
            area = '{}, Pune'.format(area_short)
            try:
                location = geolocator.geocode(area)
                latitude = location.latitude
                longitude = location.longitude
            except Exception as e:
                pass
            else:
                areas_dict['Area'].append(area_short)
                areas_dict['Latitude'].append(latitude)
                areas_dict['Longitude'].append(longitude)

    df_areas = pd.DataFrame(areas_dict)
    # df_areas.to_csv('pune_areas.csv')
    return df_areas

Load data in a dataframe and discard areas which lie in the outskirts of the city

In [5]:
# Load scraped data from csv
df_pune = pd.read_csv('pune_areas.csv')
df_pune.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
df_pune = df_pune[(df_pune['Latitude'] < 18.6) & (df_pune['Latitude'] > 18.35)
                  & (df_pune['Longitude'] < 74.0) & (df_pune['Longitude'] > 73.6)]
print(df_pune.shape)
df_pune.head(10)

(69, 3)


,Area,Latitude,Longitude
0,Aundh,18.561883,73.810196
3,Anand Nagar,18.478490,73.821326
5,Balewadi,18.582027,73.768983
6,Baner,18.564243,73.776857
7,Bund Garden Road,18.533404,73.878625
10,Budhwar Peth,18.517560,73.858037
11,Baner,18.564243,73.776857
12,Central Pune,18.520306,73.854319
17,Dhole Patil Road,18.534908,73.876258
18,Deccan Gymkhana,18.515893,73.841160


In total, there are 69 areas in Pune. The cell above shows dataframe with areas and their latitudes and longitudes

In [6]:
location = geolocator.geocode('Pune')
pune_map = folium.Map(location=[location.latitude, location.longitude], zoom_start=11)
for lat, lon, area in zip(df_pune['Latitude'], df_pune['Longitude'], df_pune['Area']):
    label = folium.Popup(area)
    folium.CircleMarker(
        [lat, lon],
        color='blue',
        popup=label,
        radius=8,
        fill_color='yellow',
        fill=True,
        fill_opacity=0.8
    ).add_to(pune_map)
pune_map

In [7]:
LIMIT=100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query=restaurant'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h4>Get list of restaurants under each area</h4>

In [8]:
pune_venues = getNearbyVenues(names=df_pune['Area'],
                                   latitudes=df_pune['Latitude'],
                                   longitudes=df_pune['Longitude']
                                  )

Aundh
Anand Nagar
Balewadi
Baner
Bund Garden Road
Budhwar Peth
Baner
Central Pune
Dhole Patil Road
Deccan Gymkhana
Dapodi
Fergusson College Road
Fursungi
Fatima Nagar
Ghorpadi
Gultekdi
Hadapsar
Hinjewadi
Hinjewadi
Kothrud
Kalyani Nagar
Karve Nagar
Koregaon Park
Kharadi
Katraj
Kondhwa
Khadki
Laxmi Road
Magarpatta
Mundhwa
Mangalwar Peth
MG Road
Mumbai - Bangalore Highway
Mundhwa - Kharadi Road
Municipal Colony
Nagar Road
Nagar - Pune Highway
New Airport Road
Nana Peth
Narayan Peth
Narhe
NIA Campus Road
Pirangut
Pune - Solapur Highway
Pimple Saudagar
Pune Central City
Pune
Ragvilas Society
Revenue Colony
Senapati Bapat Road
Shaniwar Peth
Somwar Peth
Sadashiv Peth
Sinhagad Road
Swargate
Sangamwadi Road
Shukrawar Peth
Sky Lounge
Saswad Road
Swargate
University Road
Viman Nagar
Vitthalrao Navale Rd
Vishrantwadi
Wadgaon Sheri
Wanowrie
Ward No. 8
Wagholi
Yerwada


In [9]:
print(pune_venues.shape)
pune_venues.head()

(611, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aundh,18.561883,73.810196,Picantos Mexican Grill,18.560654,73.812447,Mexican Restaurant
1,Aundh,18.561883,73.810196,Venkys Xprs,18.560550,73.808964,Fast Food Restaurant
2,Aundh,18.561883,73.810196,Café Maroo,18.564801,73.809141,Korean Restaurant
3,Aundh,18.561883,73.810196,The Loft,18.562714,73.807630,Indian Restaurant
4,Aundh,18.561883,73.810196,Hyderabadi Spice,18.560594,73.810108,Indian Restaurant


API lists down 623 total restaurants in the city of Pune.

In [10]:
# Group by neighborhood/area
pune_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Anand Nagar,8,8,8,8,8,8
Aundh,23,23,23,23,23,23
Baner,26,26,26,26,26,26
Budhwar Peth,5,5,5,5,5,5
Bund Garden Road,27,27,27,27,27,27
...,...,...,...,...,...,...
Wadgaon Sheri,3,3,3,3,3,3
Wagholi,5,5,5,5,5,5
Wanowrie,13,13,13,13,13,13


In [11]:
print('There are {} uniques categories.'.format(len(pune_venues['Venue Category'].unique())))

There are 52 uniques categories.


In [12]:
# one hot encoding
pune_onehot = pd.get_dummies(pune_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
pune_onehot['Neighborhood'] = pune_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [pune_onehot.columns[-1]] + list(pune_onehot.columns[:-1])
pune_onehot = pune_onehot[fixed_columns]

pune_onehot.head()

,Neighborhood,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Bistro,Breakfast Spot,Buffet,Burger Joint,Burrito Place,...,Pizza Place,Punjabi Restaurant,Restaurant,Sandwich Place,Seafood Restaurant,Snack Place,South Indian Restaurant,Steakhouse,Thai Restaurant,Vegetarian / Vegan Restaurant
0,Aundh,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Aundh,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Aundh,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Aundh,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Aundh,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
pune_onehot.shape

(611, 53)

In [14]:
pune_grouped = pune_onehot.groupby('Neighborhood').mean().reset_index()
pune_grouped

,Neighborhood,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Bistro,Breakfast Spot,Buffet,Burger Joint,Burrito Place,...,Pizza Place,Punjabi Restaurant,Restaurant,Sandwich Place,Seafood Restaurant,Snack Place,South Indian Restaurant,Steakhouse,Thai Restaurant,Vegetarian / Vegan Restaurant
0,Anand Nagar,0.0,0.0,0.000000,0.250000,0.000000,0.000000,0.000000,0.0,0.0,...,0.125000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
1,Aundh,0.0,0.0,0.000000,0.000000,0.000000,0.086957,0.000000,0.0,0.0,...,0.000000,0.0,0.043478,0.000000,0.000000,0.043478,0.000000,0.0,0.0,0.043478
2,Baner,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.076923,0.000000,0.153846,0.076923,0.076923,0.0,0.0,0.000000
3,Budhwar Peth,0.0,0.0,0.000000,0.200000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
4,Bund Garden Road,0.0,0.0,0.037037,0.074074,0.037037,0.037037,0.037037,0.0,0.0,...,0.074074,0.0,0.037037,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Wadgaon Sheri,0.0,0.0,0.000000,0.000000,0.000000,0.333333,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.333333,0.0,0.0,0.000000
59,Wagholi,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.200000,0.0,0.200000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.200000
60,Wanowrie,0.0,0.0,0.000000,0.153846,0.000000,0.076923,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.076923,0.000000,0.076923,0.000000,0.0,0.0,0.000000
61,Ward No. 8,0.0,0.0,0.000000,0.200000,0.000000,0.000000,0.000000,0.0,0.2,...,0.000000,0.0,0.000000,0.000000,0.000000,0.200000,0.000000,0.0,0.0,0.000000


In [15]:
pune_grouped.shape

(63, 53)

Listing down top five types of restaurants in each area

In [16]:
num_top_venues = 5

for hood in pune_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = pune_grouped[pune_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Anand Nagar----
                  venue  freq
0     Indian Restaurant  0.25
1                Bakery  0.25
2  Fast Food Restaurant  0.25
3                  Café  0.12
4           Pizza Place  0.12


----Aundh----
                  venue  freq
0     Indian Restaurant  0.22
1  Fast Food Restaurant  0.22
2        Breakfast Spot  0.09
3                  Café  0.09
4            Food Court  0.09


----Baner----
                     venue  freq
0        Indian Restaurant  0.31
1       Seafood Restaurant  0.15
2     Fast Food Restaurant  0.15
3  South Indian Restaurant  0.08
4              Snack Place  0.08


----Budhwar Peth----
                     venue  freq
0        Indian Restaurant   0.6
1        Indian Sweet Shop   0.2
2                   Bakery   0.2
3  North Indian Restaurant   0.0
4               Irani Cafe   0.0


----Bund Garden Road----
                  venue  freq
0     Indian Restaurant  0.37
1    Chinese Restaurant  0.11
2                  Café  0.07
3           Pizza Plac

                 venue  freq
0    Indian Restaurant   0.4
1               Bakery   0.2
2          Snack Place   0.2
3        Burrito Place   0.2
4  American Restaurant   0.0


----Revenue Colony----
                 venue  freq
0    Indian Restaurant   0.4
1               Bakery   0.2
2          Snack Place   0.2
3        Burrito Place   0.2
4  American Restaurant   0.0


----Sadashiv Peth----
                           venue  freq
0              Indian Restaurant  0.33
1                    Snack Place  0.20
2                 Breakfast Spot  0.13
3  Vegetarian / Vegan Restaurant  0.13
4                     Restaurant  0.07


----Sangamwadi Road----
                      venue  freq
0  Mediterranean Restaurant  0.33
1         Indian Restaurant  0.17
2        Italian Restaurant  0.17
3        Seafood Restaurant  0.17
4        Mughlai Restaurant  0.17


----Saswad Road----
                            venue  freq
0               Indian Restaurant   0.5
1                        Pet Café   0

In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create area-wise top 10 most common type of restaurants/food outlets

In [18]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = pune_grouped['Neighborhood']

for ind in np.arange(pune_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(pune_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Anand Nagar,Indian Restaurant,Bakery,Fast Food Restaurant,Pizza Place,Café,Vegetarian / Vegan Restaurant,Comfort Food Restaurant,Food Truck,Food Court,Food
1,Aundh,Fast Food Restaurant,Indian Restaurant,Food Court,Café,Breakfast Spot,Korean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Food Truck,Vegetarian / Vegan Restaurant
2,Baner,Indian Restaurant,Seafood Restaurant,Fast Food Restaurant,Café,South Indian Restaurant,Snack Place,Restaurant,Food Truck,Donut Shop,Creperie
3,Budhwar Peth,Indian Restaurant,Bakery,Indian Sweet Shop,Vegetarian / Vegan Restaurant,Comfort Food Restaurant,Food Truck,Food Court,Food,Fast Food Restaurant,Falafel Restaurant
4,Bund Garden Road,Indian Restaurant,Chinese Restaurant,Pizza Place,Café,Bakery,Fast Food Restaurant,Buffet,Kebab Restaurant,Restaurant,Breakfast Spot


Use k-means clustering method to cluster city in 3 segments

In [20]:
# set number of clusters
kclusters = 3

pune_grouped_clustering = pune_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(pune_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 2, 0, 0, 0])

In [21]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

pune_merged = df_pune

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
pune_merged = pune_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Area')

pune_merged.head() # check the last columns!

,Area,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aundh,18.561883,73.810196,0.0,Fast Food Restaurant,Indian Restaurant,Food Court,Café,Breakfast Spot,Korean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Food Truck,Vegetarian / Vegan Restaurant
3,Anand Nagar,18.478490,73.821326,0.0,Indian Restaurant,Bakery,Fast Food Restaurant,Pizza Place,Café,Vegetarian / Vegan Restaurant,Comfort Food Restaurant,Food Truck,Food Court,Food
5,Balewadi,18.582027,73.768983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Baner,18.564243,73.776857,0.0,Indian Restaurant,Seafood Restaurant,Fast Food Restaurant,Café,South Indian Restaurant,Snack Place,Restaurant,Food Truck,Donut Shop,Creperie
7,Bund Garden Road,18.533404,73.878625,0.0,Indian Restaurant,Chinese Restaurant,Pizza Place,Café,Bakery,Fast Food Restaurant,Buffet,Kebab Restaurant,Restaurant,Breakfast Spot


In [22]:
pune_merged.dropna(axis=0, inplace=True)

In [23]:
pune_merged['Cluster Labels'] = pune_merged['Cluster Labels'].astype(int)
pune_merged.head(20)

,Area,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aundh,18.561883,73.810196,0,Fast Food Restaurant,Indian Restaurant,Food Court,Café,Breakfast Spot,Korean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Food Truck,Vegetarian / Vegan Restaurant
3,Anand Nagar,18.478490,73.821326,0,Indian Restaurant,Bakery,Fast Food Restaurant,Pizza Place,Café,Vegetarian / Vegan Restaurant,Comfort Food Restaurant,Food Truck,Food Court,Food
6,Baner,18.564243,73.776857,0,Indian Restaurant,Seafood Restaurant,Fast Food Restaurant,Café,South Indian Restaurant,Snack Place,Restaurant,Food Truck,Donut Shop,Creperie
7,Bund Garden Road,18.533404,73.878625,0,Indian Restaurant,Chinese Restaurant,Pizza Place,Café,Bakery,Fast Food Restaurant,Buffet,Kebab Restaurant,Restaurant,Breakfast Spot
10,Budhwar Peth,18.517560,73.858037,0,Indian Restaurant,Bakery,Indian Sweet Shop,Vegetarian / Vegan Restaurant,Comfort Food Restaurant,Food Truck,Food Court,Food,Fast Food Restaurant,Falafel Restaurant
11,Baner,18.564243,73.776857,0,Indian Restaurant,Seafood Restaurant,Fast Food Restaurant,Café,South Indian Restaurant,Snack Place,Restaurant,Food Truck,Donut Shop,Creperie
12,Central Pune,18.520306,73.854319,0,Indian Restaurant,Snack Place,Bakery,Burrito Place,Vegetarian / Vegan Restaurant,Comfort Food Restaurant,Food Truck,Food Court,Food,Fast Food Restaurant
17,Dhole Patil Road,18.534908,73.876258,0,Indian Restaurant,Chinese Restaurant,Bakery,Café,Pizza Place,Fast Food Restaurant,Mediterranean Restaurant,Mughlai Restaurant,Kebab Restaurant,Italian Restaurant
18,Deccan Gymkhana,18.515893,73.841160,0,Indian Restaurant,Fast Food Restaurant,Restaurant,Café,Snack Place,Vegetarian / Vegan Restaurant,Sandwich Place,Pizza Place,Bakery,Bistro
21,Dapodi,18.580846,73.832775,2,Food Court,Breakfast Spot,Restaurant,Cafeteria,Chinese Restaurant,Vegetarian / Vegan Restaurant,Comfort Food Restaurant,Food Truck,Food,Fast Food Restaurant


In [24]:
# create map
map_clusters = folium.Map(location=[location.latitude, location.longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(pune_merged['Latitude'], pune_merged['Longitude'], pune_merged['Area'], pune_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [25]:
# Cluster 0
pune_merged.loc[pune_merged['Cluster Labels'] == 0, pune_merged.columns[[1] + list(range(5, pune_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,18.561883,Indian Restaurant,Food Court,Café,Breakfast Spot,Korean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Food Truck,Vegetarian / Vegan Restaurant
3,18.478490,Bakery,Fast Food Restaurant,Pizza Place,Café,Vegetarian / Vegan Restaurant,Comfort Food Restaurant,Food Truck,Food Court,Food
6,18.564243,Seafood Restaurant,Fast Food Restaurant,Café,South Indian Restaurant,Snack Place,Restaurant,Food Truck,Donut Shop,Creperie
7,18.533404,Chinese Restaurant,Pizza Place,Café,Bakery,Fast Food Restaurant,Buffet,Kebab Restaurant,Restaurant,Breakfast Spot
10,18.517560,Bakery,Indian Sweet Shop,Vegetarian / Vegan Restaurant,Comfort Food Restaurant,Food Truck,Food Court,Food,Fast Food Restaurant,Falafel Restaurant
11,18.564243,Seafood Restaurant,Fast Food Restaurant,Café,South Indian Restaurant,Snack Place,Restaurant,Food Truck,Donut Shop,Creperie
12,18.520306,Snack Place,Bakery,Burrito Place,Vegetarian / Vegan Restaurant,Comfort Food Restaurant,Food Truck,Food Court,Food,Fast Food Restaurant
17,18.534908,Chinese Restaurant,Bakery,Café,Pizza Place,Fast Food Restaurant,Mediterranean Restaurant,Mughlai Restaurant,Kebab Restaurant,Italian Restaurant
18,18.515893,Fast Food Restaurant,Restaurant,Café,Snack Place,Vegetarian / Vegan Restaurant,Sandwich Place,Pizza Place,Bakery,Bistro
23,18.524148,Asian Restaurant,Sandwich Place,Restaurant,Burger Joint,Italian Restaurant,Donut Shop,South Indian Restaurant,Seafood Restaurant,Café


In [26]:
# Cluster 1
pune_merged.loc[pune_merged['Cluster Labels'] == 1, pune_merged.columns[[1] + list(range(5, pune_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,18.526967,Vegetarian / Vegan Restaurant,Chinese Restaurant,Food Truck,Food Court,Food,Fast Food Restaurant,Falafel Restaurant,Eastern European Restaurant,Donut Shop
63,18.511282,Breakfast Spot,Café,Vegetarian / Vegan Restaurant,Comfort Food Restaurant,Food Truck,Food Court,Food,Fast Food Restaurant,Falafel Restaurant
109,18.550361,Donut Shop,Vegetarian / Vegan Restaurant,Chinese Restaurant,Food Truck,Food Court,Food,Fast Food Restaurant,Falafel Restaurant,Eastern European Restaurant


In [27]:
# Cluster 2
pune_merged.loc[pune_merged['Cluster Labels'] == 2, pune_merged.columns[[1] + list(range(5, pune_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,18.580846,Breakfast Spot,Restaurant,Cafeteria,Chinese Restaurant,Vegetarian / Vegan Restaurant,Comfort Food Restaurant,Food Truck,Food,Fast Food Restaurant
28,18.493894,Burger Joint,Vegetarian / Vegan Restaurant,Comfort Food Restaurant,French Restaurant,Food Truck,Food Court,Food,Fast Food Restaurant,Falafel Restaurant
34,18.489404,Snack Place,Bakery,Breakfast Spot,Vegetarian / Vegan Restaurant,Comfort Food Restaurant,Food Court,Food,Fast Food Restaurant,Falafel Restaurant
40,18.477746,Vegetarian / Vegan Restaurant,Thai Restaurant,Café,Pizza Place,Indian Restaurant,Bakery,BBQ Joint,Asian Restaurant,Food Court
48,18.533670,Thai Restaurant,Snack Place,Eastern European Restaurant,Chinese Restaurant,Food Truck,Food Court,Food,Fast Food Restaurant,Falafel Restaurant
56,18.557350,Thai Restaurant,BBQ Joint,Bakery,Indian Restaurant,Restaurant,Food Truck,Pizza Place,Italian Restaurant,Vegetarian / Vegan Restaurant
57,18.589521,Snack Place,Fast Food Restaurant,Restaurant,Vegetarian / Vegan Restaurant,Chinese Restaurant,Food Court,Food,Falafel Restaurant,Eastern European Restaurant
62,18.573425,Fast Food Restaurant,Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Food Truck,Food Court,Food,Falafel Restaurant,Eastern European Restaurant
79,18.525574,Restaurant,Snack Place,Sandwich Place,Pizza Place,Vegetarian / Vegan Restaurant,Chaat Place,Food,Fast Food Restaurant,Falafel Restaurant
85,18.500368,Chaat Place,Vegetarian / Vegan Restaurant,Comfort Food Restaurant,French Restaurant,Food Court,Food,Fast Food Restaurant,Falafel Restaurant,Eastern European Restaurant
